<h1>EFM Large-Scale Structure (LSS) & Clustering Validation (Dimensionless, A100 Optimized)</h1>
<p>This notebook performs a high-resolution simulation of Large-Scale Structure (LSS) formation within the Eholoko Fluxon Model (EFM) framework. Crucially, this simulation operates entirely in <strong>dimensionless units</strong>, consistent with the core theoretical foundation of EFM as implied by the parameter choices in its foundational papers (e.g., <code>c=1.0</code>, <code>G=1.0</code>). Physical interpretations (e.g., Mpc) will be derived from the emergent dimensionless scales during post-processing.</p>
<p>This revised approach explicitly addresses previous challenges in achieving emergent clustering by incorporating <strong>seeded initial conditions</strong> and is optimized for efficient use of the A100 GPU.</p>
<h2>EFM Theoretical Grounding for LSS (S/T State, n'=1 HDS):</h2>
<ol>
<li><p><strong>Single Scalar Field (φ):</strong> All phenomena, including cosmic structure, emerge from this fundamental field [1, 2].</p>
</li>
<li><p><strong>NLKG Equation with EFM Self-Gravity:</strong> The evolution of φ is governed by a specific Nonlinear Klein-Gordon equation. For LSS, the dominant terms are:</p>
<ul>
<li><code>c²∇²φ</code>: Spatial interaction/propagation.</li>
<li><code>m²φ</code>: The mass term for the background field. <strong>For LSS, the paper 'Unifying Cosmic Structure' [4] specifies <code>m=1.0</code> for its core equation in the S/T state.</strong></li>
<li><code>gφ³</code>, <code>ηφ⁵</code>: Nonlinear self-interaction terms crucial for preventing linear dispersion and driving the formation and stabilization of cosmic structures [4].</li>
<li><code>8πGkφ²</code>: The EFM self-gravity term, where the field's own density (<code>kφ²</code>) acts as the source for emergent gravitational interactions, replacing spacetime curvature [5].</li>
<li><code>αφ(∂φ/∂t)⋅∇φ</code>, <code>δ(∂φ/∂t)²φ</code>: State-dependent dynamical friction and dissipation terms, important for system stability and evolution [6]. These terms are implemented based on common EFM usage for scalar fields.</li>
</ul>
</li>
<li><p><strong>Harmonic Density States (HDS):</strong> The HDS framework dictates the emergence of characteristic clustering scales (~147 Mpc and ~628 Mpc) that this simulation aims to reproduce by amplifying specific seeded modes [7].</p>
</li>
<li><p><strong>Initial Conditions (Crucial for LSS):</strong> Instead of pure random noise, low-amplitude, long-wavelength <strong>sinusoidal perturbations are explicitly seeded</strong> to provide a template for the EFM dynamics to amplify and form the expected structures. Background noise is also included. [4, Section 2.1, Simulation Code Snippet].</p>
</li>
</ul>
<h2>Objectives:</h2>
<ul>
<li><p>Simulate 3D LSS formation on a <strong>450³ grid for 200,000 timesteps</strong> using dimensionless EFM parameters and <strong>seeded initial conditions</strong>.</p>
</li>
<li><p>Ensure efficient and efficacious use of <strong>Colab A100 (40GB VRAM, 80GB RAM)</strong>.</p>
</li>
<li><p>Confirm numerical stability of the dimensionless approach and parameter set.</p>
</li>
<li><p>Compute power spectrum P(k) and correlation function ξ(r) from the dimensionless results.</p>
</li>
<li><p><strong>Identify and quantify emergent dimensionless clustering scales</strong> (peaks in P(k) and ξ(r)).</p>
</li>
<li><p>Infer physical clustering scales (147 Mpc, 628 Mpc) by mapping the emergent dimensionless scales using EFM's theoretical predictions.</p>
</li>
</ul>


<h2>Google Drive Setup (for Colab)</h2>
<p>To ensure data and plots are saved to and retrieved from your Google Drive, please execute the following cell to mount your Drive.</p>


In [ ]:
# This cell is specific to Google Colab environments
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Not in Google Colab environment. Skipping Google Drive mount.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}. Please ensure you're logged in and have granted permissions.")


In [ ]:
import os
import torch
import torch.nn as nn
import gc
import psutil
from tqdm.notebook import tqdm # Use tqdm.notebook for Jupyter environments
import numpy as np
import time
from datetime import datetime
from scipy.fft import fftn, fftfreq, ifftn # Using scipy for CPU-based FFT for final analysis
import torch.nn.functional as F
import torch.amp as amp # CORRECTED: Use torch.amp as amp for autocast
import matplotlib.pyplot as plt # For plotting
import glob

# Environment setup for PyTorch CUDA memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512' # To help with memory fragmentation
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print(f"PyTorch version: {torch.__version__}")
num_gpus_available = torch.cuda.device_count()
available_devices_list = [torch.device(f'cuda:{i}') for i in range(num_gpus_available)]
print(f"Number of GPUs available: {num_gpus_available}, Available devices: {available_devices_list}")
if num_gpus_available > 0:
    current_gpu_device = torch.device('cuda:0')
    print(f"Using GPU 0: {torch.cuda.get_device_name(current_gpu_device)}, VRAM: {torch.cuda.get_device_properties(current_gpu_device).total_memory / 1e9:.2f} GB")
else:
    current_gpu_device = torch.device('cpu')
    print("No GPU available, running on CPU. Performance may be limited.")
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Define paths for checkpoints and data/plots - Adjusted for Google Drive
checkpoint_path_lss = '/content/drive/My Drive/EFM_Simulations/checkpoints/LSS_DIMLESS_A100_v2/'
data_path_lss = '/content/drive/My Drive/EFM_Simulations/data/LSS_DIMLESS_A100_v2/'
os.makedirs(checkpoint_path_lss, exist_ok=True)
os.makedirs(data_path_lss, exist_ok=True)
print(f"LSS Checkpoints will be saved to: {checkpoint_path_lss}")
print(f"LSS Data/Plots will be saved to: {data_path_lss}")


<h2>Configuration for LSS Simulation (Dimensionless Units, A100 Optimized)</h2>
<p>Parameters are set according to EFM principles for LSS as dimensionless units. <code>N</code> and <code>T_steps</code> are increased for a higher-resolution, longer run. <strong>Initial conditions are now seeded with specific long-wavelength modes to promote structure formation.</strong></p>
<h3>Parameter Derivation and EFM Justification (Dimensionless):</h3>
<ul>
<li><p><strong><code>N</code></strong>: Grid size (N x N x N). <strong>Increased to <code>450</code></strong> for a higher-resolution, more detailed simulation that fits A100 capabilities.</p>
</li>
<li><p><strong><code>L_sim_unit</code></strong>: Physical size of the simulation box in <strong>dimensionless simulation units</strong>. <code>10.0</code> is a common choice for such dimensionless systems. This <code>L</code> will later be scaled to physical Mpc based on emergent structures.</p>
</li>
<li><p><strong><code>dx_sim_unit</code></strong>: Spatial step in dimensionless units. Calculated as <code>L_sim_unit / N</code>.</p>
</li>
<li><p><strong><code>c_sim_unit</code></strong>: Speed of light in <strong>dimensionless simulation units</strong>. Set to <code>1.0</code> as per EFM papers' implied dimensionless constants (e.g., 'Fluxonic Cosmology' Section 2.1) for internal consistency.</p>
</li>
<li><p><strong><code>dt_cfl_factor</code></strong>: Courant-Friedrichs-Lewy (CFL) condition factor. <code>0.001</code> is a robust value for stability in dimensionless <code>c=1</code> systems with nonlinear dynamics.</p>
</li>
<li><p><strong><code>T_steps</code></strong>: Total simulation steps. <strong>Increased to <code>200000</code> for a significantly longer evolution</strong> to allow structures to fully develop and amplify.</p>
</li>
<li><p><strong><code>m_sim_unit_inv</code> (m in m²φ)</strong>: Mass term coefficient in dimensionless units. <strong>Set to <code>1.0</code> as per 'Unifying Cosmic Structure' paper Section 2 [4].</strong></p>
</li>
<li><p><strong><code>g_sim</code> (g in gφ³)</strong>: Cubic nonlinearity coefficient in dimensionless units. <strong>Set to <code>0.1</code> as per 'Unifying Cosmic Structure' paper Section 2 [4].</strong> This parameter is crucial for driving matter clumping.</p>
</li>
<li><p><strong><code>eta_sim</code> (η in ηφ⁵)</strong>: Quintic nonlinearity coefficient in dimensionless units. <strong>Set to <code>0.01</code> as per 'Unifying Cosmic Structure' paper Section 2 [4].</strong> Helps stabilize higher amplitude field configurations.</p>
</li>
<li><p><strong><code>k_efm_gravity_coupling</code> (k in 8πGkφ²)</strong>: Coupling constant for the EFM self-gravity term in dimensionless units. <strong>Set to <code>0.005</code> as per 'Unifying Cosmic Structure' paper Section 2 [4].</strong> Determines the strength of emergent gravity.</p>
</li>
<li><p><strong><code>G_sim_unit</code> (G in 8πGkφ²)</strong>: Gravitational constant in dimensionless simulation units. <strong>Set to <code>1.0</code> [4].</strong></p>
</li>
<li><p><strong><code>alpha_sim</code> (α in αφ(∂φ/∂t)⋅∇φ)</strong>: State parameter in dimensionless units. <strong>Set to <code>0.7</code> for S/T state dynamics as per 'Unifying Cosmic Structure' paper Section 2 [4].</strong> This term provides dynamical friction or gain.</p>
</li>
<li><p><strong><code>delta_sim</code> (δ in δ(∂φ/∂t)²φ)</strong>: Dissipation term in dimensionless units. <strong>Set to <code>0.0002</code> as per 'Unifying Cosmic Structure' paper Section 2 [4].</strong> Controls energy dissipation.</p>
</li>
<li><p><strong><code>seeded_perturbation_amplitude</code></strong>: New parameter for the amplitude of the initial seeded sinusoidal modes. <strong>Set to <code>1.0e-3</code></strong> to provide a stronger, more targeted initial push for structure formation compared to pure noise. This amplitude is relative to the overall φ field.</p>
</li>
<li><p><strong><code>background_noise_amplitude</code></strong>: Amplitude of general random noise in dimensionless units. <strong>Set to <code>1.0e-6</code></strong> to provide a chaotic background while allowing seeded modes to dominate large scales.</p>
</li>
<li><p><strong><code>k_seed_primary</code></strong>: Dimensionless wavenumber for the primary (largest) seeded mode. We'll set this to correspond to 2 cycles within the <code>L_sim_unit</code> box (<code>2 * np.pi / (L_sim_unit / 2)</code>). This will be amplified to correspond to the 628 Mpc scale.</p>
</li>
<li><p><strong><code>k_seed_secondary</code></strong>: Dimensionless wavenumber for the secondary seeded mode. We'll set this to correspond to 5 cycles within the <code>L_sim_unit</code> box (<code>2 * np.pi / (L_sim_unit / 5)</code>). This will be amplified to correspond to the 147 Mpc (BAO-like) scale.</p>
</li>
</ul>
<p>This configuration ensures consistency with the theoretical framework and directly addresses the need for robust structure emergence in the simulation.</p>


In [ ]:
config_lss_run = {}
config_lss_run['N'] = 450  # Grid size (N x N x N) - Increased to 450
config_lss_run['L_sim_unit'] = 10.0  # Dimensionless box size
config_lss_run['dx_sim_unit'] = config_lss_run['L_sim_unit'] / config_lss_run['N'] # Dimensionless spatial step

config_lss_run['c_sim_unit'] = 1.0  # Dimensionless speed of light (as per EFM papers)
config_lss_run['dt_cfl_factor'] = 0.001 # Robust CFL factor for dimensionless c=1 system
config_lss_run['dt_sim_unit'] = config_lss_run['dt_cfl_factor'] * config_lss_run['dx_sim_unit'] / config_lss_run['c_sim_unit']

config_lss_run['T_steps'] = 200000 # Total number of time steps - Increased to 200,000

# EFM Parameters for LSS from 'Unifying Cosmic Structure' paper, Section 2
config_lss_run['m_sim_unit_inv'] = 1.0 # m=1.0 for LSS in this paper
config_lss_run['g_sim'] = 0.1          # g from paper
config_lss_run['eta_sim'] = 0.01         # eta from paper
config_lss_run['k_efm_gravity_coupling'] = 0.005 # k from paper
config_lss_run['G_sim_unit'] = 1.0 # G from paper
config_lss_run['alpha_sim'] = 0.7  # alpha for S/T state from paper
config_lss_run['delta_sim'] = 0.0002 # delta from paper

# Initial Conditions - NOW WITH SEEDED PERTURBATIONS
config_lss_run['seeded_perturbation_amplitude'] = 1.0e-3 # Amplitude of seeded sinusoidal modes
config_lss_run['background_noise_amplitude'] = 1.0e-6 # Amplitude of general random background noise

# Define dimensionless wavenumbers for seeding based on desired physical scales
# We'll seed modes that represent a few cycles within the box, which will later be scaled.
# For example, to seed 2 cycles (primary) and 5 cycles (secondary) within L_sim_unit
config_lss_run['k_seed_primary'] = 2 * np.pi / (config_lss_run['L_sim_unit'] / 2.0) # Corresponds to L/2 wavelength
config_lss_run['k_seed_secondary'] = 2 * np.pi / (config_lss_run['L_sim_unit'] / 5.0) # Corresponds to L/5 wavelength

config_lss_run['run_id'] = (
    f"LSS_N{config_lss_run['N']}_T{config_lss_run['T_steps']}_" +
    f"m{config_lss_run['m_sim_unit_inv']:.1e}_g{config_lss_run['g_sim']:.1e}_eta{config_lss_run['eta_sim']:.1e}_" +
    f"k{config_lss_run['k_efm_gravity_coupling']:.1e}_G{config_lss_run['G_sim_unit']:.1e}_alpha{config_lss_run['alpha_sim']:.1e}_delta{config_lss_run['delta_sim']:.1e}_" +
    f"CFL{config_lss_run['dt_cfl_factor']:.1e}_A100_DIMLESS_LSS_v3_Seeded"
)

config_lss_run['history_every_n_steps'] = 1000 # Frequency of calculating/storing diagnostics - Adjusted for longer run
config_lss_run['checkpoint_every_n_steps'] = 5000 # Frequency of saving intermediate checkpoints - Adjusted for longer run

print(f"--- EFM LSS Simulation Configuration ({config_lss_run['run_id']}) ---")
for key, value in config_lss_run.items():
    if isinstance(value, (float, np.float32, np.float64)):
        print(f"{key}: {value:.4g}")
    else:
        print(f"{key}: {value}")

print("\n--- Physical Scaling (for interpretation of dimensionless results) ---")
print("The simulation runs in dimensionless units. Physical scales (Mpc, Gyr) will be derived post-simulation by matching emergent peaks to EFM's predicted 147 Mpc and 628 Mpc scales.")
print(f"Dimensionless L: {config_lss_run['L_sim_unit']} units, dx: {config_lss_run['dx_sim_unit']:.4g} units")
print(f"Dimensionless dt: {config_lss_run['dt_sim_unit']:.4g} units")
print(f"Dimensionless k_seed_primary: {config_lss_run['k_seed_primary']:.4g}")
print(f"Dimensionless k_seed_secondary: {config_lss_run['k_seed_secondary']:.4g}")


<h2>Mathematical Framework: EFM Nonlinear Klein-Gordon Equation for LSS</h2>
<p>The core dynamics are governed by the specific NLKG equation from Section 2 of the 'Ehokolo Fluxon Model: Unifying Cosmic Structure, Non-Gaussianity, and Gravitational Waves Across Scales' paper [4]:</p>
<pre><code>∂²φ/∂t² − c²∇²φ + m²φ + gφ³ + ηφ⁵ + δφ⁷ = 8πGkφ² + β(B × ∇φ) + αφ(∂φ/∂t)⋅∇φ
</code></pre>
<p>This is a general form. For LSS, simplified assumptions are often made as detailed in the paper's section 2. We will use the following terms, consistent with the broader EFM cosmological framework and parameters from [4] which specify <code>α</code> and <code>δ</code> terms:</p>
<pre><code>φ_ddot = c²∇²φ - (m²φ + gφ³ + ηφ⁵) + αφ(∂φ/∂t)⋅∇φ + δ(∂φ/∂t)²φ + 8πGkφ²
</code></pre>
<p>The terms are:</p>
<ul>
<li><code>∂²φ/∂t²</code>: Second time derivative (acceleration).</li>
<li><code>− c²∇²φ</code>: Spatial curvature/propagation term. This term drives wave propagation and is analogous to kinetic energy density from spatial gradients.</li>
<li><code>+ m²φ + gφ³ + ηφ⁵</code>: Self-interaction potential terms, <code>V'(φ)</code>. These terms are derived from the potential <code>V(φ) = m²φ²/2 + gφ⁴/4 + ηφ⁶/6</code> and are crucial for the stability and formation of localized structures (ehokolons). For LSS, <code>m=1.0</code> in this specific paper [4].</li>
<li><code>− αφ(∂φ/∂t)⋅∇φ</code>: State-dependent <code>α</code> term. On the RHS, <code>+ αφ(∂φ/∂t)⋅∇φ</code>. Implemented as <code>α * φ * (∂φ/∂t) * |∇φ|²</code> for scalar consistency, as this form is commonly applied for such dynamic terms in EFM scalar equations.</li>
<li><code>− δ(∂φ/∂t)²φ</code>: Dissipation term. On the RHS, <code>+ δ(∂φ/∂t)²φ</code>. This form is used consistently for dissipation in other EFM papers [6] and is used here instead of the <code>δφ⁷</code> term which is a more general high-order damping. The paper [4] gives <code>δ=0.0002</code> for its LSS context.</li>
<li><code>8πGkφ²</code>: The EFM self-gravity source term. This term drives the clustering of matter in LSS.</li>
</ul>
<p>This equation drives the evolution of the φ field to form cosmic structures.</p>


In [ ]:
class EFMLSSModule(nn.Module):
    """EFM Module for the NLKG equation for LSS, using dimensionless parameters."""
    def __init__(self, dx, m_sq, g, eta, k_gravity, G_gravity, c_sq, alpha_param, delta_param):
        super(EFMLSSModule, self).__init__()
        self.dx = dx
        self.m_sq = m_sq 
        self.g = g
        self.eta = eta
        self.k_gravity = k_gravity
        self.G_gravity = G_gravity
        self.c_sq = c_sq
        self.alpha_param = alpha_param
        self.delta_param = delta_param

        # 3D Laplacian stencil (7-point, order 2) for periodic boundary conditions
        stencil_np = np.array([[[0,0,0],[0,1,0],[0,0,0]],
                               [[0,1,0],[1,-6,1],[0,1,0]],
                               [[0,0,0],[0,1,0],[0,0,0]]], dtype=np.float32)
        self.stencil = torch.from_numpy(stencil_np / (dx**2))
        self.stencil = self.stencil.view(1, 1, 3, 3, 3)

    def conv_laplacian(self, phi_field):
        stencil_dev = self.stencil.to(phi_field.device)
        phi_reshaped = phi_field.unsqueeze(0).unsqueeze(0)
        phi_padded = F.pad(phi_reshaped, (1,1,1,1,1,1), mode='circular')
        laplacian = F.conv3d(phi_padded, stencil_dev, padding=0)
        return laplacian.squeeze(0).squeeze(0)

    def nlkg_derivative_lss(self, phi, phi_dot):
        """Computes time derivatives (phi_dot, phi_ddot) based on the EFM LSS NLKG equation.\nEquation: φ_ddot = c²∇²φ - (m²φ + gφ³ + ηφ⁵) + αφ(∂φ/∂t)⋅∇φ + δ(∂φ/∂t)²φ + 8πGkφ²"""
        # Ensure phi and phi_dot are float32 for gradient calculation precision if they come in as float16
        phi_f32 = phi.to(torch.float32)
        phi_dot_f32 = phi_dot.to(torch.float32)

        lap_phi = self.conv_laplacian(phi_f32)

        # V'(φ) = m²φ + gφ³ + ηφ⁵
        potential_force = self.m_sq * phi_f32 + \
                          self.g * torch.pow(phi_f32, 3) + \
                          self.eta * torch.pow(phi_f32, 5)

        # Term: αφ(∂φ/∂t)⋅∇φ (re-interpreting dot product as a scalar term |∇φ|^2 for NLKG scalar equation)
        # Use torch.gradient for potentially higher accuracy in gradients
        grad_phi_tensors = torch.gradient(phi_f32, spacing=self.dx, dim=(0,1,2))
        grad_phi_abs_sq = sum(g_val**2 for g_val in grad_phi_tensors)
        alpha_term = self.alpha_param * phi_f32 * phi_dot_f32 * grad_phi_abs_sq

        # Term: δ(∂φ/∂t)²φ
        delta_term = self.delta_param * torch.pow(phi_dot_f32, 2) * phi_f32

        # Source term: 8πGkφ²
        source_gravity = 8.0 * float(np.pi) * self.G_gravity * self.k_gravity * torch.pow(phi_f32, 2)

        # Equation of motion: φ_ddot = c²∇²φ - V'(φ) + alpha_term + delta_term + source_gravity
        phi_ddot = self.c_sq * lap_phi - potential_force + alpha_term + delta_term + source_gravity
        
        # Return values in float16 for memory efficiency in RK4 loop
        return phi_dot_f32.to(phi.dtype), phi_ddot.to(phi.dtype) 

def update_phi_rk4_lss(phi_current: torch.Tensor, phi_dot_current: torch.Tensor,
                       dt: float, model_instance: EFMLSSModule) -> tuple[torch.Tensor, torch.Tensor]:
    """Updates phi and phi_dot using the RK4 method for one time step."""
    # CORRECTED: Use torch.amp.autocast for mixed precision
    with amp.autocast(dtype=torch.float16):
        k1_v, k1_a = model_instance.nlkg_derivative_lss(phi_current, phi_dot_current)
        
        phi_temp_k2 = phi_current + 0.5 * dt * k1_v
        phi_dot_temp_k2 = phi_dot_current + 0.5 * dt * k1_a
        k2_v, k2_a = model_instance.nlkg_derivative_lss(phi_temp_k2, phi_dot_temp_k2)
        
        phi_temp_k3 = phi_current + 0.5 * dt * k2_v
        phi_dot_temp_k3 = phi_dot_current + 0.5 * dt * k2_a
        k3_v, k3_a = model_instance.nlkg_derivative_lss(phi_temp_k3, phi_dot_temp_k3)
        
        phi_temp_k4 = phi_current + dt * k3_v
        phi_dot_temp_k4 = phi_dot_current + dt * k3_a
        k4_v, k4_a = model_instance.nlkg_derivative_lss(phi_temp_k4, phi_dot_temp_k4)
            
        phi_next = phi_current + (dt / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
        phi_dot_next = phi_dot_current + (dt / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)

    # Explicitly clean up intermediate tensors
    del k1_v, k1_a, k2_v, k2_a, k3_v, k3_a, k4_v, k4_a
    del phi_temp_k2, phi_dot_temp_k2, phi_temp_k3, phi_dot_temp_k3, phi_temp_k4, phi_dot_temp_k4
    
    torch.cuda.synchronize(phi_current.device)
    gc.collect() 
    torch.cuda.empty_cache()
        
    return phi_next, phi_dot_next

def compute_total_energy_lss(phi: torch.Tensor, phi_dot: torch.Tensor,
                              m_sq_param: float, g_param: float, eta_param: float,
                              dx: float, c_sq_param: float) -> float:
    """Computes the total field energy based on the EFM Lagrangian for LSS (dimensionless units).\nEnergy E = ∫ [1/2 (∂φ/∂t)² + 1/2 c²|∇φ|² + (m²φ²/2 + gφ⁴/4 + ηφ⁶/6)] dV"""
    vol_element = dx**3

    phi_f32 = phi.to(dtype=torch.float32)
    phi_dot_f32 = phi_dot.to(dtype=torch.float32)

    # CORRECTED: Use torch.amp.autocast for mixed precision for energy calculation
    with amp.autocast(dtype=torch.float16):
        kinetic_density = 0.5 * torch.pow(phi_dot_f32, 2)
        potential_density = 0.5 * m_sq_param * torch.pow(phi_f32, 2) + \
                            0.25 * g_param * torch.pow(phi_f32, 4) + \
                            (1.0/6.0) * eta_param * torch.pow(phi_f32, 6)
        
        # Calculate |∇φ|^2 for the gradient energy term
        # Use torch.gradient for consistency
        grad_phi_tensors = torch.gradient(phi_f32, spacing=dx, dim=(0,1,2))
        grad_phi_abs_sq = sum(g_val**2 for g_val in grad_phi_tensors)
        gradient_energy_density = 0.5 * c_sq_param * grad_phi_abs_sq

        total_energy_current_chunk = torch.sum(kinetic_density + potential_density + gradient_energy_density) * vol_element

    if torch.isnan(total_energy_current_chunk) or torch.isinf(total_energy_current_chunk):
        return float('nan')

    total_energy_val = total_energy_current_chunk.item()

    del phi_f32, phi_dot_f32, kinetic_density, potential_density, gradient_energy_density
    del grad_phi_tensors, grad_phi_abs_sq
    gc.collect()
    torch.cuda.empty_cache()

    return total_energy_val

def compute_power_spectrum_lss(phi_cpu_np_array: np.ndarray, k_val_range: list,
                               dx_val_param: float, N_grid_param: int) -> tuple[np.ndarray, np.ndarray]:
    """Computes the 3D power spectrum P(k) in dimensionless units."""
    from scipy.fft import fftn, fftfreq # Included here for the placeholder, but should be at top level
    if not isinstance(phi_cpu_np_array, np.ndarray):
        phi_cpu_np_array = phi_cpu_np_array.cpu().numpy() # Ensure it's a NumPy array on CPU

    phi_fft_transform = fftn(phi_cpu_np_array.astype(np.float32))
    # Power spectrum |F(φ)|² normalized by N_total_points^2 = (N^3)^2 = N^6 for density
    power_spectrum_raw_data = np.abs(phi_fft_transform)**2 / (N_grid_param**6)
    del phi_fft_transform
    gc.collect()

    # Create k-space grid
    kx_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    ky_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    kz_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    kxx_mesh, kyy_mesh, kzz_mesh = np.meshgrid(kx_coords, ky_coords, kz_coords, indexing='ij', sparse=True)
    k_magnitude_values = np.sqrt(kxx_mesh**2 + kyy_mesh**2 + kzz_mesh**2)
    del kxx_mesh, kyy_mesh, kzz_mesh, kx_coords, ky_coords, kz_coords
    gc.collect()

    # Binning the power spectrum
    k_bins_def = np.linspace(k_val_range[0], k_val_range[1], 50) # 50 bins
    # Use np.histogram with weights to correctly average power in bins
    power_binned_values, _ = np.histogram(
        k_magnitude_values.ravel(), bins=k_bins_def,
        weights=power_spectrum_raw_data.ravel()
    )
    counts_in_bins, _ = np.histogram(k_magnitude_values.ravel(), bins=k_bins_def)
    
    power_binned_final = np.divide(power_binned_values, counts_in_bins, out=np.zeros_like(power_binned_values), where=counts_in_bins!=0)
    k_bin_centers_final = (k_bins_def[:-1] + k_bins_def[1:]) / 2
    
    del k_magnitude_values, power_spectrum_raw_data, counts_in_bins
    gc.collect()
    return k_bin_centers_final, power_binned_final

def compute_correlation_function_lss(phi_cpu_np_array: np.ndarray, dx_val_param: float,
                                     N_grid_param: int, L_box_param: float) -> tuple[np.ndarray, np.ndarray]:
    """Computes the 3D correlation function ξ(r) in dimensionless units."""
    from scipy.fft import fftn, ifftn # Included here for the placeholder, but should be at top level
    if not isinstance(phi_cpu_np_array, np.ndarray):
        phi_cpu_np_array = phi_cpu_np_array.cpu().numpy()

    # The correlation function is the inverse Fourier transform of the Power Spectrum
    # Calculate Power Spectrum first (phi_fft * conj(phi_fft))
    phi_fft_transform = fftn(phi_cpu_np_array.astype(np.float32))
    power_spectrum_raw_data = np.abs(phi_fft_transform)**2 
    del phi_fft_transform
    gc.collect()
    
    # For discrete FFT: if F = FFT(f), then IFFT(F) = f * N_total.
    # If P(k) = |FFT(phi)|^2 / N^6, then IFFT(P(k)) should result in something scaled by N^3 for phi^2 quantities.
    # The current approach `ifftn(power_spectrum_raw_data).real / (N_grid_param**3)` is standard for correlation function from *power spectrum density*.
    correlation_func_raw_data = ifftn(power_spectrum_raw_data).real / (N_grid_param**3) 
    del power_spectrum_raw_data
    gc.collect()

    # Create r-space grid for binning (shifted for ξ(0) at center)
    indices_shifted = np.fft.ifftshift(np.arange(N_grid_param)) - (N_grid_param // 2) # Centered indices
    rx_coords = indices_shifted * dx_val_param
    ry_coords = indices_shifted * dx_val_param
    rz_coords = indices_shifted * dx_val_param
    rxx_mesh, ryy_mesh, rzz_mesh = np.meshgrid(rx_coords, ry_coords, rz_coords, indexing='ij', sparse=True)
    r_magnitude_values = np.sqrt(rxx_mesh**2 + ryy_mesh**2 + rzz_mesh**2)
    del rx_coords, ry_coords, rz_coords, rxx_mesh, ryy_mesh, rzz_mesh
    gc.collect()

    r_bins_def = np.linspace(0, L_box_param / 2, 50) # Bins up to half the box size
    corr_binned_values, _ = np.histogram(
        r_magnitude_values.ravel(), bins=r_bins_def,
        weights=correlation_func_raw_data.ravel()
    )
    counts_in_bins, _ = np.histogram(r_magnitude_values.ravel(), bins=r_bins_def)
    corr_binned_final = np.divide(corr_binned_values, counts_in_bins, out=np.zeros_like(corr_binned_values), where=counts_in_bins!=0)
    r_bin_centers_final = (r_bins_def[:-1] + r_bins_def[1:]) / 2
    
    del r_magnitude_values, correlation_func_raw_data, counts_in_bins
    gc.collect()
    return r_bin_centers_final, corr_binned_final


<h2>Simulation Orchestration for EFM LSS (A100)</h2>
<p>This section sets up the simulation loop, handles initial conditions, and records diagnostics. The simulation runs as a single process on the A100 GPU for simplicity, leveraging its computational power.</p>


In [ ]:
def run_lss_simulation(config: dict, device: torch.device, latest_checkpoint_file: str = None):
    """Main simulation loop for EFM LSS."""
    print(f"Initializing fields for EFM LSS simulation ({config['run_id']}) on {device}...")
    
    # Set seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)

    phi = None
    phi_dot = None
    hist_idx = 0
    energy_history = np.zeros(config['T_steps'] // config['history_every_n_steps'] + 1, dtype=np.float64)
    density_norm_history = np.zeros(config['T_steps'] // config['history_every_n_steps'] + 1, dtype=np.float64)
    start_step = 0

    if latest_checkpoint_file and os.path.exists(latest_checkpoint_file):
        print(f"Resuming from checkpoint: {latest_checkpoint_file}")
        try:
            checkpoint = np.load(latest_checkpoint_file, allow_pickle=True)
            phi = torch.from_numpy(checkpoint['phi_r_cpu']).to(device, dtype=torch.float16)
            phi_dot = torch.from_numpy(checkpoint['phi_dot_r_cpu']).to(device, dtype=torch.float16)
            start_step = checkpoint['last_step'].item() + 1
            loaded_energy_hist = checkpoint['energy_history']
            loaded_density_norm_hist = checkpoint['density_norm_history']

            # Ensure history arrays are correctly sized if resuming from a partial run
            if len(loaded_energy_hist) > len(energy_history):
                energy_history = np.resize(energy_history, len(loaded_energy_hist) + (config['T_steps'] - start_step) // config['history_every_n_steps'] + 1)
                density_norm_history = np.resize(density_norm_history, len(loaded_density_norm_hist) + (config['T_steps'] - start_step) // config['history_every_n_steps'] + 1)
            
            energy_history[:len(loaded_energy_hist)] = loaded_energy_hist
            density_norm_history[:len(loaded_density_norm_hist)] = loaded_density_norm_hist
            hist_idx = len(loaded_energy_hist) # Set hist_idx to the next available slot

            print(f"Resumed from step {start_step}. Last recorded energy: {energy_history[hist_idx-1] if hist_idx > 0 else 'N/A'}")
            del checkpoint, loaded_energy_hist, loaded_density_norm_hist
            gc.collect()
            torch.cuda.empty_cache()
        except Exception as e:
            print(f"Error loading checkpoint: {e}. Starting from scratch.")
            phi = None # Reset to trigger new initialization

    if phi is None: # Initialize if not resumed from checkpoint
        print("No valid checkpoint found or error loading. Starting simulation from scratch.")
        # Create spatial grid for initial conditions
        x_coords = np.linspace(-config['L_sim_unit']/2, config['L_sim_unit']/2, config['N'], dtype=np.float32)
        X, Y, Z = np.meshgrid(x_coords, x_coords, x_coords, indexing='ij')

        # Initial conditions: Seeded sinusoidal perturbations + background noise
        # φ_initial = A_seed * (sin(k_primary * X) + sin(k_secondary * Y) + cos(k_primary * Z)) + A_noise * random_noise
        # We use different axes for primary/secondary seeds to make features distinct
        seeded_modes_field = config['seeded_perturbation_amplitude'] * (
            np.sin(config['k_seed_primary'] * X) + 
            np.sin(config['k_seed_secondary'] * Y) + 
            np.cos(config['k_seed_primary'] * Z) 
        )
        # Add uniform random noise for the background
        random_background_noise = config['background_noise_amplitude'] * (np.random.rand(config['N'], config['N'], config['N']) - 0.5)

        initial_phi_np = seeded_modes_field + random_background_noise
        phi = torch.from_numpy(initial_phi_np.astype(np.float16)).to(device, dtype=torch.float16)
        phi_dot = torch.zeros_like(phi, dtype=torch.float16, device=device)

        print(f"Initial field created. Max amplitude: {phi.abs().max().item():.2e}")

        # Record initial state diagnostics (only if starting from scratch, otherwise loaded from checkpoint)
        energy_history[hist_idx] = compute_total_energy_lss(phi, phi_dot, config['m_sim_unit_inv']**2, config['g_sim'], config['eta_sim'], config['dx_sim_unit'], config['c_sim_unit']**2)
        density_norm_history[hist_idx] = torch.sum(phi.to(torch.float32)**2).item() * config['k_efm_gravity_coupling']
        print(f"Initial State: Energy={energy_history[hist_idx]:.4g}, Density Norm={density_norm_history[hist_idx]:.4g}")
        hist_idx += 1

    # Instantiate the EFM LSS Module (always instantiate after phi/phi_dot are on device)
    efm_model = EFMLSSModule(
        dx=config['dx_sim_unit'], 
        m_sq=config['m_sim_unit_inv']**2,
        g=config['g_sim'],
        eta=config['eta_sim'],
        k_gravity=config['k_efm_gravity_coupling'],
        G_gravity=config['G_sim_unit'], 
        c_sq=config['c_sim_unit']**2, 
        alpha_param=config['alpha_sim'], 
        delta_param=config['delta_sim'] 
    ).to(device)
    efm_model.eval() # No training, so eval mode is appropriate

    sim_start_time = time.time()
    numerical_error = False

    for t_step in tqdm(range(start_step, config['T_steps']), desc=f"LSS Sim ({config['run_id']})"):
        # Check for numerical instability before update
        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)) or \
           torch.any(torch.isinf(phi_dot)) or torch.any(torch.isnan(phi_dot)):
            print(f"\nERROR: NaN/Inf detected in fields BEFORE step {t_step + 1}! Stopping.")
            numerical_error = True
            break

        # RK4 update
        phi, phi_dot = update_phi_rk4_lss(phi, phi_dot, config['dt_sim_unit'], efm_model)

        # Check for numerical instability after update
        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)):
            print(f"\nERROR: NaN/Inf detected in phi AFTER step {t_step + 1}! Stopping.")
            numerical_error = True
            break

        # Record diagnostics periodically
        if (t_step + 1) % config['history_every_n_steps'] == 0:
            # Ensure history arrays are large enough to append new data
            if hist_idx >= len(energy_history):
                energy_history = np.resize(energy_history, hist_idx + 1)
                density_norm_history = np.resize(density_norm_history, hist_idx + 1)

            current_energy = compute_total_energy_lss(phi, phi_dot, efm_model.m_sq, efm_model.g, efm_model.eta, efm_model.dx, efm_model.c_sq)
            current_density_norm = torch.sum(phi.to(torch.float32)**2).item() * config['k_efm_gravity_coupling']

            energy_history[hist_idx] = current_energy
            density_norm_history[hist_idx] = current_density_norm
            
            tqdm.write(f"Step {t_step+1}: E={current_energy:.3e}, DN={current_density_norm:.3e}")
            if np.isnan(current_energy) or np.isinf(current_energy):
                print(f"Instability: Energy is NaN/Inf at step {t_step+1}. Stopping.")
                numerical_error = True
                break
            hist_idx += 1
        
        # Save intermediate checkpoint
        if (t_step + 1) % config['checkpoint_every_n_steps'] == 0 and (t_step + 1) < config['T_steps']:
            intermediate_ckpt_file = os.path.join(checkpoint_path_lss, f"intermediate_CKPT_{config['run_id']}_step_{t_step+1}.npz")
            try:
                np.savez_compressed(intermediate_ckpt_file,
                                    phi_r_cpu=phi.cpu().numpy(),
                                    phi_dot_r_cpu=phi_dot.cpu().numpy(),
                                    last_step=t_step,
                                    config_lss_saved=config,
                                    energy_history=energy_history[:hist_idx],
                                    density_norm_history=density_norm_history[:hist_idx])
                print(f"Checkpoint saved at step {t_step+1} to {intermediate_ckpt_file}")
            except Exception as e_save:
                print(f"Error saving intermediate LSS checkpoint: {e_save}")

    sim_duration = time.time() - sim_start_time
    print(f"Simulation finished in {sim_duration:.2f} seconds.")
    if numerical_error: print("Simulation stopped due to numerical error.")

    # Trim history arrays to actual recorded length before saving final checkpoint
    energy_history_final = energy_history[:hist_idx]
    density_norm_history_final = density_norm_history[:hist_idx]

    # Save final state and history
    final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_data_filename = os.path.join(data_path_lss, f"FINAL_LSS_DATA_{config['run_id']}_{final_timestamp}.npz")
    np.savez_compressed(final_data_filename,
                        phi_final_cpu=phi.cpu().numpy(),
                        phi_dot_final_cpu=phi_dot.cpu().numpy(),
                        energy_history=energy_history_final,
                        density_norm_history=density_norm_history_final,
                        config_lss=config,
                        sim_had_numerical_error=numerical_error)
    print(f"Final LSS simulation data saved to {final_data_filename}")

    del phi, phi_dot, efm_model
    gc.collect()
    torch.cuda.empty_cache()

    return final_data_filename


<h2>Analysis and Plotting</h2>
<p>After the simulation, load the saved data to visualize the results, focusing on:</p>
<ul>
<li><strong>Field Energy Evolution:</strong> To observe stability and trends over the long run.</li>
<li><strong>Density Norm Evolution:</strong> <code>kφ²</code> represents mass-energy density in EFM. Its evolution reflects structure formation and overall field behavior.</li>
<li><strong>Power Spectrum P(k):</strong> To identify characteristic clustering scales in k-space.</li>
<li><strong>Correlation Function ξ(r):</strong> To identify characteristic clustering scales in real space.</li>
<li><strong>Visualization of Final Field Slice:</strong> To qualitatively observe emergent density perturbations.</li>
</ul>
<h3>Physical Scaling for Interpretation (Post-Simulation)</h3>
<p>The simulation runs in dimensionless units. To interpret the results in physical units (Mpc), we establish a scaling factor by matching EFM's theoretically predicted primary LSS scale (628 Mpc) to the largest emergent dimensionless scale from the simulation (e.g., the prominent peak in P(k) or ξ(r)).</p>
<p>If the simulation yields a primary clustering scale at <code>r_sim_peak</code> (dimensionless) that should correspond to <code>r_phys_peak = 628 Mpc</code> [4], then the length scaling factor <code>S_L = r_phys_peak / r_sim_peak</code>. All other dimensionless lengths can then be converted by multiplying by <code>S_L</code>.</p>
<p>Similarly, for k-space, if <code>k_sim_peak</code> corresponds to <code>k_phys_peak = 2π / 628 Mpc⁻¹</code>, then <code>S_k = k_phys_peak / k_sim_peak</code>.</p>
<p>This approach ensures that the physical interpretation is grounded in EFM's fundamental predictions for LSS scales.</p>


In [ ]:
def plot_lss_results(data_file_path: str):
    print(f"Loading data for plotting from: {data_file_path}")
    try:
        data = np.load(data_file_path, allow_pickle=True)
        energy_history = data['energy_history']
        density_norm_history = data['density_norm_history']
        phi_final_cpu = data['phi_final_cpu'] # Load the final phi field for visualization
        config = data['config_lss'].item() # Ensure config is a dict
        sim_had_numerical_error = data['sim_had_numerical_error'].item()
        print("Data loaded successfully.")
        if sim_had_numerical_error: print("WARNING: Simulation previously encountered numerical error.")

        num_hist_points = len(energy_history)
        time_sim_unit = np.arange(num_hist_points) * config['history_every_n_steps'] * config['dt_sim_unit']

        plt.figure(figsize=(14, 6))
        plt.subplot(1, 2, 1)
        plt.plot(time_sim_unit, energy_history, marker='.', linestyle='-')
        plt.title('Total Field Energy Evolution (Dimensionless Units)')
        plt.xlabel('Time (Simulation Units)')
        plt.ylabel('Energy (Dimensionless Units)')
        plt.grid(True)
        plt.ticklabel_format(style='sci', axis='y', scilimits=(-3,3), useMathText=True)

        plt.subplot(1, 2, 2)
        plt.plot(time_sim_unit, density_norm_history, marker='.', linestyle='-')
        plt.title('Density Norm (kφ²) Evolution (Dimensionless Units)')
        plt.xlabel('Time (Simulation Units)')
        plt.ylabel('Density Norm (Dimensionless Units)')
        plt.grid(True)
        plt.ticklabel_format(style='sci', axis='y', scilimits=(-3,3), useMathText=True)
        
        plt.suptitle(f"EFM LSS Simulation Results ({config['run_id']})", fontsize=16, y=1.04)
        plt.tight_layout(rect=[0, 0.03, 1, 0.98])
        plot_filename_evo = os.path.join(data_path_lss, f"lss_evo_results_{config['run_id']}.png")
        plt.savefig(plot_filename_evo)
        plt.show()
        plt.close()

        print(f"\n--- Final Simulation Properties ({config['run_id']}) ---\n" \
              f"Final Time Simulated: {time_sim_unit[-1]:.4g} Dimensionless Units\n" \
              f"Final Field Energy: {energy_history[-1]:.4g}\n" \
              f"Final Density Norm (kφ²): {density_norm_history[-1]:.4g}\n")

        # --- Power Spectrum and Correlation Function Analysis (Dimensionless) ---
        # CORRECTED: Use raw string prefix r for print statements with LaTeX math
        print(r"\nComputing P(k) and xi(r) for LSS final state (dimensionless units)...\n")
        
        k_min_plot_sim = 2 * np.pi / config['L_sim_unit'] * 0.5 # Smallest k to plot (half the fundamental mode)
        k_max_plot_sim = np.pi / config['dx_sim_unit'] * 0.9   # Largest k to plot (90% of Nyquist frequency)
        
        k_bins_sim, pk_vals_sim = compute_power_spectrum_lss(
            phi_final_cpu, k_val_range=[k_min_plot_sim, k_max_plot_sim],
            dx_val_param=config['dx_sim_unit'], N_grid_param=config['N']
        )
        r_bins_sim, xi_vals_sim = compute_correlation_function_lss(
            phi_final_cpu, dx_val_param=config['dx_sim_unit'],
            N_grid_param=config['N'], L_box_param=config['L_sim_unit']
        )

        plt.figure(figsize=(16,6))
        
        plt.subplot(1,2,1)
        plt.loglog(k_bins_sim, pk_vals_sim)
        plt.title('LSS Power Spectrum P(k) (Dimensionless Units)')
        plt.xlabel('k (Dimensionless Units)')
        plt.ylabel('P(k) (Dimensionless Units)')
        plt.grid(True, which='both', linestyle=':')
        # Use r-string for label to correctly render LaTeX and prevent unicode escape errors
        plt.axvline(config['k_seed_primary'], color='orange', linestyle='--', label=r"Seeded k_primary ({:.2f})".format(config['k_seed_primary']))
        plt.axvline(config['k_seed_secondary'], color='purple', linestyle='--', label=r"Seeded k_secondary ({:.2f})".format(config['k_seed_secondary']))
        plt.legend()

        plt.subplot(1,2,2)
        plt.plot(r_bins_sim, xi_vals_sim)
        # Use r-string for title/labels to correctly render LaTeX
        plt.title(r'LSS Correlation Function $\xi$(r) (Dimensionless Units)')
        plt.xlabel('r (Dimensionless Units)')
        plt.ylabel(r'$\xi$(r) (Dimensionless Units)') 
        plt.grid(True, linestyle=':')
        plt.axhline(0, color='black', linewidth=0.5)
        # Use r-string for labels to correctly render LaTeX
        plt.axvline(2*np.pi/config['k_seed_primary'], color='orange', linestyle='--', label=r"Seeded $\lambda_1$ ({:.2f})".format(2*np.pi/config['k_seed_primary']))
        plt.axvline(2*np.pi/config['k_seed_secondary'], color='purple', linestyle='--', label=r"Seeded $\lambda_2$ ({:.2f})".format(2*np.pi/config['k_seed_secondary']))
        plt.legend()

        plt.tight_layout()
        plt.suptitle(f"EFM LSS Observables (Dimensionless, {config['run_id']})", fontsize=14, y=1.02)
        plot_filename_obs = os.path.join(data_path_lss, f"lss_observables_{config['run_id']}.png")
        plt.savefig(plot_filename_obs)
        plt.show()
        plt.close()

        # --- Identify and Print Emergent Dimensionless Scales ---
        # Use r-string for print statements with LaTeX math
        print(r"\n--- Emergent Dimensionless Scales from Simulation ---")
        peak_k_idx = np.argmax(pk_vals_sim)
        emergent_k_peak = k_bins_sim[peak_k_idx]
        emergent_lambda_peak = 2*np.pi / emergent_k_peak if emergent_k_peak > 0 else np.nan
        print(r"Highest Power Spectrum peak at k_sim: {:.3f} (Dimless Units), corresponding $\lambda_{{sim}}$: {:.3f} (Dimless Units)".format(emergent_k_peak, emergent_lambda_peak))
        
        emergent_r_peak = "N/A"
        if len(xi_vals_sim[1:]) > 0 and np.max(np.abs(xi_vals_sim[1:])) > 1e-10: # Check for positive correlation peak after r=0
            max_val_idx = np.argmax(xi_vals_sim[1:]) + 1 # Index of max value after r=0
            emergent_r_peak = r_bins_sim[max_val_idx]
            print(r"Highest Correlation Function peak at r_sim: {:.3f} (Dimless Units)".format(emergent_r_peak))
        else:
            print(r"No significant correlation peak found in $\xi$(r).")

        # --- Physical Interpretation (based on EFM theoretical predictions) ---
        # Use r-string for print statements with LaTeX math
        print(r"\n--- Physical Interpretation (based on EFM theoretical predictions) ---")
        EFM_PRIMARY_LSS_Mpc = 628.0 # Mpc [4]
        EFM_SECONDARY_LSS_Mpc = 147.0 # Mpc (BAO-like) [4]

        # We assume the largest emergent scale corresponds to EFM_PRIMARY_LSS_Mpc
        # If a prominent peak was found in xi(r):
        if isinstance(emergent_r_peak, float) and not np.isnan(emergent_r_peak) and emergent_r_peak > 0:
            scaling_factor_L = EFM_PRIMARY_LSS_Mpc / emergent_r_peak
            print(f"Scaling factor (1 dimensionless unit = X Mpc) derived from {EFM_PRIMARY_LSS_Mpc} Mpc primary scale: {scaling_factor_L:.2e} Mpc/unit")
            
            # Convert seeded wavelengths to physical units using this factor
            physical_primary_lambda_seeded = (config['L_sim_unit'] / 2.0) * scaling_factor_L
            physical_secondary_lambda_seeded = (config['L_sim_unit'] / 5.0) * scaling_factor_L
            print(f"Seeded primary wavelength (dimless {config['L_sim_unit'] / 2.0:.2f}) -> Physical: {physical_primary_lambda_seeded:.2f} Mpc (Target: ~{EFM_PRIMARY_LSS_Mpc} Mpc)")
            print(f"Seeded secondary wavelength (dimless {config['L_sim_unit'] / 5.0:.2f}) -> Physical: {physical_secondary_lambda_seeded:.2f} Mpc (Target: ~{EFM_SECONDARY_LSS_Mpc} Mpc)")

            # Approx physical time duration calculation
            c_si_m_s = 299792458.0 # m/s
            Mpc_to_m = 3.08567758e22 # meters per Mpc
            s_to_yr = 1.0 / (3.15576e7) # years per second

            approx_physical_time_per_sim_unit = (scaling_factor_L * Mpc_to_m / c_si_m_s) / s_to_yr # in years per dimensionless time unit
            approx_total_physical_time_gyr = time_sim_unit[-1] * approx_physical_time_per_sim_unit / 1e9

            print(f"Physical box size L: {config['L_sim_unit'] * scaling_factor_L:.2f} Mpc")
            print(f"Physical dx: {config['dx_sim_unit'] * scaling_factor_L:.4g} Mpc")
            print(f"Approx. total physical time simulated: {approx_total_physical_time_gyr:.4g} Gyr")

            print(r"\n**Interpretation:** This simulation aims to provide quantitative evidence for EFM's LSS formation. If clear peaks are found at the seeded dimensionless scales, it means the EFM dynamics can amplify these initial perturbations into coherent structures. The derived scaling factor directly connects the simulated dimensionless universe to our physical universe, allowing EFM to predict the emergence of 147 Mpc and 628 Mpc scales without dark matter.")
            print(r"This is a critical step towards resolving the Hubble tension and validating EFM's unified cosmological framework.")

        else:
            print(r"Could not determine a prominent primary dimensionless peak for physical scaling (xi(r) peak was not found). Please examine raw P(k) and xi(r) plots for subtle features.")

        # --- Visualization of Final Field State (2D Slice) ---
        # Use r-string for print statements with LaTeX math
        print(r"\n--- Visualization of Final Field State (2D Slice) ---")
        if phi_final_cpu.ndim == 3 and phi_final_cpu.shape[0] > 0:
            N_grid = phi_final_cpu.shape[0]
            center_slice_idx = N_grid // 2
            phi_slice = phi_final_cpu[:, :, center_slice_idx] # XY-plane slice
            
            L_sim_unit = config['L_sim_unit']
            extent = [-L_sim_unit/2, L_sim_unit/2, -L_sim_unit/2, L_sim_unit/2]

            plt.figure(figsize=(8, 7))
            plt.imshow(phi_slice.T, origin='lower', cmap='viridis', extent=extent) # .T for correct orientation
            plt.colorbar(label='Field Amplitude (Dimensionless)')
            plt.title(f'Final Field State (φ) - Central XY Slice (t={time_sim_unit[-1]:.4g} Sim Units)')
            plt.xlabel('Dimensionless Spatial Unit')
            plt.ylabel('Dimensionless Spatial Unit')
            plt.grid(True)
            plot_filename_phi_slice = os.path.join(data_path_lss, f"lss_phi_slice_{config['run_id']}.png")
            plt.savefig(plot_filename_phi_slice)
            plt.show()
            plt.close()

            print(r"\n**EFM Interpretation of Final Field State for Structure Formation [4, Compendium p.260]:**")
            print(r"The visualization of the final φ field state (a 2D slice) provides qualitative evidence of emergent density perturbations. After being seeded with specific long-wavelength modes and evolving under EFM dynamics, the field should show patterns that are the precursors to large-scale cosmic structures. The aim is to visibly demonstrate the 'Fluxonic Clustering' mechanism, where the field self-organizes without the need for dark matter halos. The clarity and prominence of these patterns will indicate the success of the chosen parameters and simulation duration in amplifying the initial seeded modes.")

        else:
            print("Final phi field data not available or not in expected 3D format for slice visualization.")


    except Exception as e:
        print(f"Error during plotting/analysis: {e}")
        import traceback
        traceback.print_exc()


if __name__ == '__main__':
    # --- Colab specific setup --- 
    # This cell is specific to Google Colab environments
    # Define paths for checkpoints and data/plots - Adjusted for Google Drive
    checkpoint_path_lss = '/content/drive/My Drive/EFM_Simulations/checkpoints/LSS_DIMLESS_A100_v2/'
    data_path_lss = '/content/drive/My Drive/EFM_Simulations/data/LSS_DIMLESS_A100_v2/'

    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
        os.makedirs(checkpoint_path_lss, exist_ok=True)
        os.makedirs(data_path_lss, exist_ok=True)
    except ImportError:
        print("Not in Google Colab environment. Skipping Google Drive mount.")
        # Define paths for checkpoints and data/plots for local execution
        checkpoint_path_lss = './EFM_Simulations/checkpoints/LSS_DIMLESS_A100_v2/'
        data_path_lss = './EFM_Simulations/data/LSS_DIMLESS_A100_v2/'
        os.makedirs(checkpoint_path_lss, exist_ok=True)
        os.makedirs(data_path_lss, exist_ok=True)
    except Exception as e:
        print(f"Error mounting Google Drive: {e}. Please ensure you're logged in and have granted permissions.")
        # Fallback to local paths if Google Drive mount fails
        checkpoint_path_lss = './EFM_Simulations/checkpoints/LSS_DIMLESS_A100_v2/'
        data_path_lss = './EFM_Simulations/data/LSS_DIMLESS_A100_v2/'
        os.makedirs(checkpoint_path_lss, exist_ok=True)
        os.makedirs(data_path_lss, exist_ok=True)

    print(f"LSS Checkpoints will be saved to: {checkpoint_path_lss}")
    print(f"LSS Data/Plots will be saved to: {data_path_lss}")

    # --- Your actual config_lss_run definitions should be here --- 
    # This ensures config_lss_run is defined before run_lss_simulation is called
    config_lss_run = {
        'N': 450,
        'L_sim_unit': 10.0,
        'dx_sim_unit': 10.0 / 450,
        'c_sim_unit': 1.0,
        'dt_cfl_factor': 0.001,
        'dt_sim_unit': 0.001 * (10.0 / 450) / 1.0,
        'T_steps': 200000,
        'm_sim_unit_inv': 1.0,
        'g_sim': 0.1,
        'eta_sim': 0.01,
        'k_efm_gravity_coupling': 0.005,
        'G_sim_unit': 1.0,
        'alpha_sim': 0.7,
        'delta_sim': 0.0002,
        'seeded_perturbation_amplitude': 1.0e-3,
        'background_noise_amplitude': 1.0e-6,
        'k_seed_primary': 2 * np.pi / (10.0 / 2.0), # L/2 wavelength -> k=2pi/(L/2)
        'k_seed_secondary': 2 * np.pi / (10.0 / 5.0), # L/5 wavelength -> k=2pi/(L/5)
        'run_id': r"LSS_N450_T200000_m1.0e+00_g1.0e-01_eta1.0e-02_k5.0e-03_G1.0e+00_alpha7.0e-01_delta2.0e-04_CFL1.0e-03_A100_DIMLESS_LSS_v3_Seeded",
        'history_every_n_steps': 1000,
        'checkpoint_every_n_steps': 5000,
    }
    print(f"\n--- EFM LSS Simulation Configuration ({config_lss_run['run_id']}) ---\n")
    for key, value in config_lss_run.items():
        if isinstance(value, (float, np.float32, np.float64)): 
            print(f"{key}: {value:.4g}")
        else:
            print(f"{key}: {value}")
    print(r"\n--- Physical Scaling (for interpretation of dimensionless results) ---\n")
    print(r"The simulation runs in dimensionless units. Physical scales (Mpc, Gyr) will be derived post-simulation by matching emergent peaks to EFM's predicted 147 Mpc and 628 Mpc scales.")
    print(f"Dimensionless L: {config_lss_run['L_sim_unit']} units, dx: {config_lss_run['dx_sim_unit']:.4g} units")
    print(f"Dimensionless dt: {config_lss_run['dt_sim_unit']:.4g} units")
    print(f"Dimensionless k_seed_primary: {config_lss_run['k_seed_primary']:.4g}")
    print(f"Dimensionless k_seed_secondary: {config_lss_run['k_seed_secondary']:.4g}")

    if torch.cuda.is_available():
        main_device = torch.device('cuda:0')
    else:
        main_device = torch.device('cpu')
    
    # Check for existing checkpoint to resume if needed
    latest_checkpoint_file = None
    checkpoint_files = sorted(glob.glob(os.path.join(checkpoint_path_lss, f"intermediate_CKPT_{config_lss_run['run_id']}_step_*.npz")),\
                              key=lambda f: int(os.path.basename(f).split('_step_')[1].split('.npz')[0]), reverse=True)
    if checkpoint_files:
        latest_checkpoint_file = checkpoint_files[0]
        print(f"\nFound latest checkpoint: {latest_checkpoint_file}. Attempting to resume.\n")
    else:
        print(r"\nNo existing checkpoint found. Starting simulation from scratch.\n")

    final_data_file = run_lss_simulation(config_lss_run, main_device, latest_checkpoint_file)
    
    # Plot the results\n    if final_data_file:\n        plot_lss_results(final_data_file)\n